# AIHUB의 주차 공간 탐색을 위한 차량 관점 복합 데이터 전처리 과정

데이터 셋은 크게 **Training Set, Validation Set**으로 나뉩니다.

그리고 각각 원천 데이터 **(이미지 데이터)** 와 라벨링 데이터 **(json 파일)** 이 포함되어 있습니다.

본 팀의 경우 목적은 하나의 데이터셋에서 특정한 비율로 training set과 validation set을 추출해 사용하는 것이기 때문에

최대한 많은 기본 데이터셋을 확보하기 위해, 우선적으로 기존에 training과 validation으로 구분되어 있던 데이터를 하나의 데이터셋으로 통일시킵니다.




**[목차]**
* **0. AIHUBSHELL 커맨드를 사용해서 원하는 Dataset 다운로드**
* **1. 다운로드 받은 데이터셋에서 segmentation 정보를 가지고 있는 데이터만 검출**
* **2. 팀원 각자 전처리 한 데이터 셋을 하나의 데이터 셋으로 통합**
* **3. 전처리 된 segmentation json파일을 coco json 포맷으로 변경**
* **4. coco dataset의 포맷으로 변경된 json 파일들을 학습에 쓰일 json파일로 하나로 통합**

## 0. AIHUBSHELL 커맨드를 사용해서 원하는 Dataset 다운로드

* 참고 : https://www.aihub.or.kr/devsport/apishell/list.do?currMenu=403&topMenu=100

In [ ]:
#주의 : 총 데이터 290.36GB 하드디스크의 여유공간을 확인하고 실행할 것.
!aihubshell -mode d -datasetkey 598

## 1. 다운로드 받은 데이터 셋에서 segmentation 정보를 가지고 있는 데이터만 검출

### 1.1 로그 기록 파일 생성

In [ ]:
import os
import json
import shutil
import re
import cv2
import numpy as np

In [79]:
# 기록 파일 경로
progress_file = 'progress_log.txt' # 진행 상황 저장 파일
counter_file = 'counter_log.txt'  # 카운터(검출된 데이터 쌍의 갯수) 저장 파일

In [80]:
def load_processed_files(progress_file):
    """이전에 처리된 파일 목록을 로드합니다."""
    if os.path.exists(progress_file):
        with open(progress_file, 'r') as f:
            processed_files = set(line.strip() for line in f)
    else:
        processed_files = set()
    return processed_files

In [81]:
def save_processed_file(progress_file, filename):
    """처리된 파일 이름을 기록합니다."""
    with open(progress_file, 'a') as f:
        f.write(filename + '\n')

In [82]:
def load_counter(counter_file):
    """이전에 사용된 파일 카운터를 로드합니다."""
    if os.path.exists(counter_file):
        with open(counter_file, 'r') as f:
            counter = int(f.read().strip())
    else:
        counter = 1
    return counter

In [83]:
def save_counter(counter_file, counter):
    """파일 카운터를 저장합니다."""
    with open(counter_file, 'w') as f:
        f.write(str(counter))

### 1.2 파일 맵핑

지정된 라벨링데이터(json데이터) 파일의 경로의 이름을 입력받아, 그와 매칭되는 원천데이터(이미지데이터) 이름을 리턴받습니다.

In [84]:
#label_path의 라벨데이터와 연결되어 있는 원천데이터의 경로를 반환한다

def map_label_to_image_path(label_path):
    #라벨 경로의 경로 문자열을 TL, VL을 TS, VS로, label을 Camera로 바꾼다
    if "TL" in label_path:
        return label_path.replace("TL", "TS").replace("label", "Camera")
    elif "VL" in label_path:
        return label_path.replace("VL", "VS").replace("label", "Camera")
    # 위 조건에 맞지 않는 경우 원본 경로 그대로 반환
    return label_path

**함수 설명**

* os.makedirs() : 지정한 디렉토리(경로)에 새 폴더를 생성
* os.path.relpath(path, start) : start 경로에서 path 경로를 바라본 상대적인 경로를 계산
* os.path.join(a, b) : a 문자열과 b 문자열을 하나로 묶어서 반환
* os.path.normpath(path) : path 내의 ., .., // 등의 불필요한 요소를 제거 
* os.path.exits(path) : path의 존재유무에 True, False 반환

* **os.walk(path) : 주어진 path의 하위 폴더와 파일을 탐색하는 함수**

    * subdir : ./test/ 아래의 모든 폴더의 경로 리스트 반환
    * _ : subdir 경로에 하위 폴더가 또 있는 경우 그 폴더 경로의 리스트 반환
    * files : subdir 경로의 하위 파일 경로를 리스트로 반환



In [85]:
def extract_segmented_data(label_root_dir, image_root_dir, output_json_dir, output_image_dir, progress_file,
                           counter_file):
    """라벨링 데이터와 원천 데이터를 각각 탐색하여 JSON과 이미지 파일을 매칭합니다."""

    # 이전에 처리된 파일 목록을 로드합니다.
    processed_files = load_processed_files(progress_file)

    # 이전 카운터를 로드합니다.
    file_counter = load_counter(counter_file)

    # 출력 디렉토리가 존재하지 않으면 생성합니다.
    os.makedirs(output_json_dir, exist_ok=True)
    os.makedirs(output_image_dir, exist_ok=True)

    # 라벨링 데이터 디렉토리를 순회하며 JSON 파일을 찾습니다.
    for subdir, _, files in os.walk(label_root_dir):
        for json_file in files:
            if json_file.endswith(".json"):
                json_path = os.path.join(subdir, json_file)

                # 이미 처리된 파일은 건너뜁니다.
                if json_path in processed_files:
                    continue

                with open(json_path, "r") as f:
                    data = json.load(f)

                if "segmentation" in data and data["segmentation"]:
                    # 대응하는 이미지 파일 경로 생성
                    relative_path = os.path.relpath(json_path, label_root_dir)
                    mapped_relative_path = map_label_to_image_path(relative_path)
                    image_path = os.path.join(image_root_dir, mapped_relative_path.replace(".json", ".jpg"))
                    image_path = os.path.normpath(image_path)  # 경로 정규화

                    # 이미지 파일이 존재하는 경우에만 JSON과 이미지를 복사합니다.
                    if os.path.exists(image_path):
                        new_json_name = f"image_{file_counter}.json"
                        new_img_name = f"image_{file_counter}.jpg"

                        shutil.copy(json_path, os.path.join(output_json_dir, new_json_name))
                        shutil.copy(image_path, os.path.join(output_image_dir, new_img_name))

                        # 파일 처리 후 진행 상태를 기록합니다.
                        save_processed_file(progress_file, json_path)

                        # 카운터 증가 및 저장
                        file_counter += 1
                        save_counter(counter_file, file_counter)
                    else:
                        print(f"이미지 {image_path}를 찾을 수 없어서 JSON 파일 {json_file}과 연관된 파일을 건너뜁니다.")



세부 설명
* 라벨데이터와 원천데이터 경로를 입력합니다
* segmentation 정보를 기준으로 전처리 된 데이터의 반환 경로를 입력합니다

In [87]:
# json 파일 내부에 segmentation 정보가 있는 json과 그에 연결되는 이미지를 지정된 경로에 복사
label_root_dir = r'C:\Users\TRUENEAT0223\Documents\vscode\project01_local\2.Validation\라벨링데이터'  # 라벨링 데이터 루트 디렉토리
image_root_dir = r'C:\Users\TRUENEAT0223\Documents\vscode\project01_local\2.Validation\원천데이터'  # 원천 데이터 루트 디렉토리
output_json_dir = r'C:\Users\TRUENEAT0223\Documents\vscode\project01_local\segmentjson'  # 출력 JSON 파일 디렉토리
output_image_dir = r'C:\Users\TRUENEAT0223\Documents\vscode\project01_local\segmentimage'  # 출력 이미지 파일 디렉토리

extract_segmented_data(label_root_dir, image_root_dir, output_json_dir, output_image_dir, progress_file, counter_file)

## 2. 팀원 각자 전처리 한 데이터셋을 하나의 데이터셋으로 통합

세부 설명
* 병합된 json 파일들과 image 파일들의 디렉토리의 경로를 입력합니다
* 병합할 json 파일들과 image 파일들의 팀원 각각의 경로를 입력합니다

In [ ]:
# 병합될 디렉토리 경로
merged_json_dir = "/home/elicer/merged_data/merged_segmentjson"
merged_image_dir = "/home/elicer/merged_data/merged_segmentimage"


# 동료별 디렉토리 설정(인원 수에 따라 딕셔너리 키, 밸류 값 제거 가능)
collab_dirs = {
    "MSB": {
        "json_dir": "/home/elicer/MSB/segmentjson",
        "image_dir": "/home/elicer/MSB/segmentimage"
    },
    "KSH": {
        "json_dir": "/home/elicer/KSH/segmentjson",
        "image_dir": "/home/elicer/KSH/segmentimage"
    },
    "LYJ": {
        "json_dir": "/home/elicer/LYJ/segmentjson",
        "image_dir": "/home/elicer/LYJ/segmentimage"
    },
    "BDN": {
        "json_dir": "/home/elicer/LYJ/segmentjson",
        "image_dir": "/home/elicer/LYJ/segmentimage"
    }
}

함수 설명
* 병합 데이터가 모일 디렉토리 내에서 가장 큰 넘버를 return 하는 함수입니다

In [ ]:
def get_last_number(directory, extension):
    
    max_num = 0
    pattern = re.compile(r'image_(\d+)\.' + extension)  # 숫자를 포함하는 패턴 찾기
    for filename in os.listdir(directory):
        match = pattern.match(filename)
        if match:
            num = int(match.group(1))
            if num > max_num:
                max_num = num
    return max_num

함수 설명
* 병합 데이터가 모일 디렉토리에 있는 파일을 카운팅하고, 가장 큰 넘버 이후의 넘버로 데이터 복사합니다

* 예시) 마석빈 : 1, 2, 3  김서희 : 1, 2, 3    박달님 : 1, 2, 3    이윤제 : 1, 2, 3    =>  병합 데이터 : 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12

In [ ]:
# 병합 디렉토리가 존재하지 않으면 생성
os.makedirs(merged_json_dir, exist_ok=True)
os.makedirs(merged_image_dir, exist_ok=True)


# 기존 데이터에서 가장 큰 번호 찾기
current_max_num = max(
    get_last_number(merged_json_dir, "json"),
    get_last_number(merged_image_dir, "jpg")
)


# 동료들의 파일을 병합
for collab, dirs in collab_dirs.items():
    json_dir = dirs['json_dir']
    image_dir = dirs['image_dir']

    json_files = [f for f in os.listdir(json_dir) if f.endswith(".json")]
    for json_file in json_files:
        # JSON 파일과 대응되는 이미지 파일 이름 구하기
        image_file = json_file.replace(".json", ".jpg")

        json_path = os.path.join(json_dir, json_file)
        image_path = os.path.join(image_dir, image_file)

        # 이미지와 JSON 파일이 모두 존재하는지 확인
        if os.path.exists(image_path):
            current_max_num += 1
            new_json_name = f"image_{current_max_num}.json"
            new_image_name = f"image_{current_max_num}.jpg"

            # 파일 복사
            new_json_path = os.path.join(merged_json_dir, new_json_name)
            new_image_path = os.path.join(merged_image_dir, new_image_name)

            shutil.copy(json_path, new_json_path)
            shutil.copy(image_path, new_image_path)
        else:
            print(f"이미지 {image_file}를 찾을 수 없어서 JSON 파일 {json_file}과 연관된 파일을 건너뜁니다.")

print("모든 파일이 병합되었습니다.")

## 3. 전처리 된 segmentation json파일을 coco json 포맷으로 변경

함수 설명
* segmentation 영역의 넓이를 계산합니다

    Shoelace formula : 신발끈 공식으로 불리며, 좌표평면상 점의 좌표를 이용하여 다각형의 넓이를 계산하는 공식입니다. 

    |x1 * y2 + x2 * y3 + ... + xn * y1) + (y1 * x2 - y2 * x3 - ... - yn * x1| / 2

* segmentation 폴리곤들의 최소점 x, y와 최대점 x, y를 구해서 가장 넓은 bounding box 를 구합니다

In [ ]:
# 전처리 된 json파일에서 segmentation 넓이 계산
def calculate_area(polygon):
    #1차원 polygon 리스트의 자표를 2칸씩 건너뛰며 x, y좌표를 받아오기
    x = np.array(polygon[::2])
    y = np.array(polygon[1::2])

    return 0.5 * np.abs(np.dot(x, np.roll(y, 1)) - np.dot(y, np.roll(x, 1)))


# 전처리 된 json파일에서 segmentation을 기준으로 최대 bbox 계산
def calculate_bbox(polygon):
    x = polygon[::2]
    y = polygon[1::2]

    return [min(x), min(y), max(x) - min(x), max(y) - min(y)]

세부 설명
* aihub 데이터셋의 json 과 coco 데이터 포맷 json의 차이점은, segmentation의 영역을 지정할 때 사용하는 polygon을 2차원 배열로 배치하느냐 1차원 배열로 배치하느냐의 차이입니다.
* 2차원 배열로 [x1, y1], [x2, y2], ... 로 저장되어 있는 segmentation polygon 정보를 [x1, y1, x2, y2, ...] 형태로 풀어 저장합니다.

함수 설명
* coco라는 딕셔너리를 하나 만듭니다.
*  기존 segmentation 전처리 파일에서 연결되는 이미지, annotations, categories 정보를 가져와 coco에 입력합니다. 
* 이후 json.dump() 함수를 사용해 새로운 coco 데이터 포맷 json 파일을 생성합니다.

In [ ]:
def convert_json_to_coco(input_json_path, image_dir, output_dir):

    # 전처리된 json 파일 내용을 data에 입력
    with open(input_json_path, "r") as f:
        data = json.load(f)


    #input_json_path 라벨 이름 문자열에서 .json을 .jpg로 바꿔 이미지 이름으로 사용
    img_filename = os.path.basename(input_json_path).replace(".json", ".jpg")

    # 이미지 크기 고정 (1920x1080)
    width = 1920
    height = 1080


    # 틀이 되는 coco 포맷 딕셔너리
    coco = {
        "images": [{
            "id": 0,
            "file_name": img_filename, # 연결될 이미지 이름 저장
            "width": width,
            "height": height
        }],
        "annotations": [],
        "categories": [
            {"id": 1, "name": "Parking Space"},
            {"id": 2, "name": "Driveable Space"}
        ]
    }

    annotation_id = 0
    category_id_map = {cat['name']: cat['id'] for cat in coco['categories']}

    # 세그멘테이션 어노테이션 추가
    for segmentation in data.get("segmentation", []):

        # 전처리된 json파일에서 카테고리 이름 저장
        category_name = segmentation["name"]
        if category_name not in category_id_map:
            continue  # 'Parking Space'나 'Driveable Space'가 아니면 무시

        
        #'Parking Space'나 'Driveable Space'라면 그에 맞는 id 저장
        category_id = category_id_map[category_name]


        # 폴리곤 좌표를 1차원 리스트로 변환
        # polygon은 segmentation 영역의 좌표를 1차원 리스트에 풀어낸 것
        polygon = [coord for point in segmentation["polygon"] for coord in point]


        # annotation값을 정리해 딕셔너리에 입력
        annotation = {
            "id": annotation_id,
            "image_id": 0,
            "category_id": category_id,
            "segmentation": [polygon],
            "area": calculate_area(polygon),
            "bbox": calculate_bbox(polygon),
            "iscrowd": 0
        }

        # 틀이 되는 coco 포맷의 annotations 항목에 annotation 딕셔너리 저장
        coco["annotations"].append(annotation)
        annotation_id += 1

    # 파일명 생성 및 저장
    output_filename = os.path.basename(input_json_path).replace(".json", "_coco.json")
    output_path = os.path.join(output_dir, output_filename)


    # 정보를 전부 입력받은 coco 딕셔너리를 새로운 _coco.json 파일에 저장
    with open(output_path, "w") as f:
        json.dump(coco, f, indent=4) #indent=4 : 4개의 공백으로 들여쓰기

In [ ]:
# 경로 입력

input_json_dir = "/home/elicer/merged_data/merged_segmentjson"  # JSON 파일들이 있는 디렉토리
image_dir = "/home/elicer/merged_data/merged_segmentimage"  # 이미지 파일들이 있는 디렉토리
output_dir = "/home/elicer/merged_data/merged_coco_files"  # COCO 포맷으로 저장할 디렉토리

In [ ]:
# 코드 실행

os.makedirs(output_dir, exist_ok=True)

for filename in os.listdir(input_json_dir):
    if filename.endswith(".json"):
        convert_json_to_coco(os.path.join(input_json_dir, filename), image_dir, output_dir)

## 4. coco dataset의 포맷으로 변경된 json 파일들을 학습에 쓰일 json파일로 하나로 통합

* coco 포맷 데이터들을 하나의 coco 포맷 데이터로 합치는 것

예) image_1_coco.json, image_2_coco.json, image_3_coco.json, ... image_n_coco.json =>  merged_coco.json 

함수 설명
* merged_coco 라는 딕셔너리를 생성합니다.
* 새롭게 정립할 id를 annotation_id = 1, image_id = 1로 초기화 시킵니다.
* 이하 for문은 중복되는 categories, images, annotations 데이터를 새로운 id를 기준으로 중복 없이 merged_coco에 넣는 과정입니다.
* for문이 끝난 후, json.dump() 함수를 사용해 하나의 큰 coco 데이터 포맷의 json파일을 생성합니다.

In [ ]:
def merge_coco_jsons(input_dir, output_file):
    #큰 틀의 image, annotations, categories 딕셔너리
    merged_coco = {
        "images": [],
        "annotations": [],
        "categories": []
    }

    # id 초기화에 필요한 id 시작 값
    annotation_id = 1
    image_id = 1
    category_map = {}

    # input_dir에 있는 파일 하나를 filename으로 받기
    for filename in os.listdir(input_dir):
        # filename의 어미에 _coco.json이 있는가?(coco 포맷 json 파일인가?)
        if filename.endswith("_coco.json"):
            # coco json 파일을 열어서 값을 coco_data에 저장
            with open(os.path.join(input_dir, filename), 'r', encoding='utf-8') as f:
                coco_data = json.load(f)

                # 각 파일의 categories 섹션을 탐색하며, 새로운 카테고리가 있을 경우 category_map에 추가합니다. 이후 값을 merged_coco에 추가합니다.
                for category in coco_data['categories']:
                    if category['id'] not in category_map:   
                        category_map[category['id']] = len(category_map) + 1    
                        category['id'] = category_map[category['id']]
                        merged_coco['categories'].append(category)                  

                # 각 파일의 images 섹션에서 이미지를 하나씩 읽어 image_id를 새로운 ID로 재지정합니다. 이후 값을 merged_coco에 추가합니다.
                for image in coco_data['images']:       
                    image['id'] = image_id
                    merged_coco['images'].append(image)
                    image_id += 1

                # 각 파일의 annotations 섹션에서 어노테이션을 하나씩 읽은 후 새로운 annotation_id를 부여, 어노테이션의 image_id도 통합된 이미지의 ID와 일치하도록 업데이트합니다.
                for annotation in coco_data['annotations']:
                    annotation['id'] = annotation_id
                    annotation['image_id'] = image_id - 1
                    annotation['category_id'] = category_map[annotation['category_id']]
                    merged_coco['annotations'].append(annotation)
                    annotation_id += 1

    # 위의 merged_coco 딕셔너리를 json 파일로 가공
    with open(output_file, 'w', encoding='utf-8') as f:
        json.dump(merged_coco, f, indent=4)

세부 설명
* input_dir에 통합시킬 coco 데이터 포맷 json 폴더 경로를, output_file에 병합된 파일이 저장될 위치를 입력합니다.

In [ ]:

input_dir = '/home/elicer/merged_data/merged_coco_files'          # 통합시킬 coco 포맷 json 디렉토리
output_file = '/home/elicer/merged_data/merged_coco.json'   # 통합된 merge_coco.json 파일을 넣을 디렉토리 

merge_coco_jsons(input_dir, output_file)